[If set up is done](#section2)
### Setup

In [6]:
!wget -O kafka_2.12-2.4.0.tgz http://mirror.dkd.de/apache/kafka/2.4.0/kafka_2.12-2.4.0.tgz 

--2020-01-27 20:58:01--  http://mirror.dkd.de/apache/kafka/2.4.0/kafka_2.12-2.4.0.tgz
Resolving mirror.dkd.de (mirror.dkd.de)... 5.9.59.115
Connecting to mirror.dkd.de (mirror.dkd.de)|5.9.59.115|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62283588 (59M) [application/x-gzip]
Saving to: ‘kafka_2.12-2.4.0.tgz’

kafka_2.12-2.4.0.tg 100%[===================>]  59,40M   692KB/s    in 89s     

2020-01-27 20:59:45 (684 KB/s) - ‘kafka_2.12-2.4.0.tgz’ saved [62283588/62283588]



In [7]:
!tar xvzf kafka_2.12-2.4.0.tgz

kafka_2.12-2.4.0/
kafka_2.12-2.4.0/LICENSE
kafka_2.12-2.4.0/NOTICE
kafka_2.12-2.4.0/bin/
kafka_2.12-2.4.0/bin/kafka-delete-records.sh
kafka_2.12-2.4.0/bin/trogdor.sh
kafka_2.12-2.4.0/bin/kafka-preferred-replica-election.sh
kafka_2.12-2.4.0/bin/connect-mirror-maker.sh
kafka_2.12-2.4.0/bin/kafka-console-consumer.sh
kafka_2.12-2.4.0/bin/kafka-consumer-perf-test.sh
kafka_2.12-2.4.0/bin/kafka-log-dirs.sh
kafka_2.12-2.4.0/bin/zookeeper-server-stop.sh
kafka_2.12-2.4.0/bin/kafka-verifiable-consumer.sh
kafka_2.12-2.4.0/bin/kafka-acls.sh
kafka_2.12-2.4.0/bin/zookeeper-server-start.sh
kafka_2.12-2.4.0/bin/kafka-server-stop.sh
kafka_2.12-2.4.0/bin/kafka-configs.sh
kafka_2.12-2.4.0/bin/kafka-reassign-partitions.sh
kafka_2.12-2.4.0/bin/kafka-leader-election.sh
kafka_2.12-2.4.0/bin/kafka-producer-perf-test.sh
kafka_2.12-2.4.0/bin/kafka-topics.sh
kafka_2.12-2.4.0/bin/connect-standalone.sh
kafka_2.12-2.4.0/bin/kafka-dump-log.sh
kafka_2.12-2.4.0/bin/kafka-broker-api-versions.sh
kafka_2.12-2.4.0/bin/kafk

kafka_2.12-2.4.0/libs/plexus-utils-3.2.0.jar
kafka_2.12-2.4.0/libs/commons-lang3-3.8.1.jar
kafka_2.12-2.4.0/libs/connect-file-2.4.0.jar
kafka_2.12-2.4.0/libs/connect-basic-auth-extension-2.4.0.jar
kafka_2.12-2.4.0/libs/connect-mirror-2.4.0.jar
kafka_2.12-2.4.0/libs/connect-mirror-client-2.4.0.jar
kafka_2.12-2.4.0/libs/kafka-streams-2.4.0.jar
kafka_2.12-2.4.0/libs/rocksdbjni-5.18.3.jar
kafka_2.12-2.4.0/libs/kafka-streams-scala_2.12-2.4.0.jar
kafka_2.12-2.4.0/libs/kafka-streams-test-utils-2.4.0.jar
kafka_2.12-2.4.0/libs/kafka-streams-examples-2.4.0.jar


In [15]:
pip install confluent-kafka

     |████████████████████████████████| 9.1MB 736kB/s eta 0:00:01     |████████▎                       | 2.4MB 635kB/s eta 0:00:11     |█████████████████████████████▎  | 8.3MB 729kB/s eta 0:00:02     |██████████████████████████████▏ | 8.6MB 729kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install kafka-python

  Using cached https://files.pythonhosted.org/packages/49/c9/9863483a1353700ba87821b4f39085eb18fd1bcbb1e954c697177d67f03f/kafka_python-1.4.7-py2.py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


<a id=section2></a>
### Run from terminal as back ground processes not supported from here  
* cd /home/kalinga/GIT/DS/stream-processing/kafka_2.12-2.4.0/  
* bin/zookeeper-server-start.sh config/zookeeper.properties &
* bin/kafka-server-start.sh config/server.properties &  

In [2]:
cd /home/kalinga/GIT/DS/stream-processing/kafka_2.12-2.4.0/ 

/home/kalinga/GIT/DS/stream-processing/kafka_2.12-2.4.0


In [4]:
!bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic mock-data

In [3]:
!bin/kafka-topics.sh --list --bootstrap-server localhost:9092

In [18]:
!bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic mock-data

Topic mock-data is marked for deletion.
Note: This will have no impact if delete.topic.enable is not set to true.


<a id='section3'> </a>
### Python based Kafka Producer

In [6]:
from confluent_kafka import Producer
import json

In [7]:
kafka_brokers='localhost:9092'
p = Producer({
               'bootstrap.servers': kafka_brokers,
               'queue.buffering.max.messages': 100000,
               'queue.buffering.max.ms': 500, #
               'batch.num.messages': 100 #
             }) if kafka_brokers else None


In [8]:
topic='mock-data'

In [11]:
data = '/home/kalinga/GIT/DS/stream-processing/mock_data.json'

# read mock_data
with open(data, 'r') as datafile:
    json_data=datafile.read()
    
# parse file
items = json.loads(json_data)

In [17]:
for item in items:
    try:        
        if p is not None:
            print("No of msg waiting to be delivered to broker:" + str(p.__len__()))
            # produce() is asynchronous
            # enqueue the message on an internal queue
            # flush() after each produce() effectively a sync producer 
            p.produce(topic, item.encode('utf-8'))

            p.poll(0)
        else:
            print("Producer is not created!!!")
            exit(1)
    except KeyboardInterrupt:
            if p is not None:
                p.flush()
    except BufferError as e:
        print(e, file=sys.stderr)
        print("POLL for 5 seconds")

        # putting the poll() block until there is queue space available. blocks for 10 seconds
        # block for as long as makes sense for your application 
        # (preferably longer) since message delivery can take some time if there are temporary errors on the broker (e.g., leader failover)
        p.poll(5)
        # data for which there was exception, not yet added to the internal Queue
        p.produce(topic, data.encode('utf-8'))
    finally:
        if p is not None:
            p.flush()


No of msg waiting to be delivered to broker:0


AttributeError: 'dict' object has no attribute 'encode'